### Importo Librerias

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

### Cargo el Dataset

In [14]:
def load_data(df):
    texts = list(df['texto'].str.lower())
    labels = df['sentimiento'].tolist()
    return texts, labels

In [40]:
data_file = "./data.csv"
data = pd.read_csv(data_file)


In [41]:
data = data.dropna(subset=['texto'])

In [42]:
data['sentimiento'] = data['sentimiento'].replace('Indefinido', 'Neutral')

In [43]:
tf_count = data.value_counts("sentimiento")
tf_count

sentimiento
Negativo    5798
Positivo    5680
Neutral     5591
Name: count, dtype: int64

In [44]:
data = data[data['sentimiento'] != "Neutral"]

In [47]:
data['sentimiento'] = data['sentimiento'].replace({'Positivo':1, 'Negativo':0})

In [48]:
tf_count = data.value_counts("sentimiento")
tf_count

sentimiento
0    5798
1    5680
Name: count, dtype: int64

In [49]:
texts, labels = load_data(data)

In [50]:
class TextClassificationDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_length):
          self.texts = texts
          self.labels = labels
          self.tokenizer = tokenizer
          self.max_length = max_length
  def __len__(self):
      return len(self.texts)
  def __getitem__(self, idx):
      text = self.texts[idx]
      label = self.labels[idx]
      encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
      return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}


### Defino Modelo

In [4]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            pooled_output = outputs.pooler_output
            x = self.dropout(pooled_output)
            logits = self.fc(x)
            return logits

In [5]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

In [6]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions,digits=4)


In [67]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        print(outputs)
        _, preds = torch.max(outputs, dim=1)
        print(torch.max(outputs, dim=1))
        return "Positive" if preds.item() == 1 else "Negative"

### Parametros


In [75]:
# Set up parameters
bert_model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
num_classes = 2
max_length = 128
batch_size = 16
num_epochs = 10
learning_rate = 3e-5

### Divido los Dataset en Train, Valid y Test

In [71]:
train_texts, test_val_texts, train_labels, test_val_labels = train_test_split(texts, labels, test_size=0.4, random_state=42)
test_texts, val_texts, test_labels, val_labels = train_test_split(test_val_texts, test_val_labels, test_size=0.5, random_state=42)

# Imprime el tamaño de cada conjunto
print("Tamaño del conjunto de entrenamiento:", len(train_texts), len(train_labels) )
print("Tamaño del conjunto de prueba:", len(test_texts), len(test_labels))
print("Tamaño del conjunto de validación:", len(val_texts), len(val_labels))

Tamaño del conjunto de entrenamiento: 6886 6886
Tamaño del conjunto de prueba: 2296 2296
Tamaño del conjunto de validación: 2296 2296


### Tokenizer

In [59]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

In [72]:
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
test_dataset = TextClassificationDataset(test_texts, test_labels, tokenizer, max_length)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

### Cargo Modelo

In [23]:
print(torch.cuda.is_available())

True


In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [74]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Entreno Modelo

In [76]:
for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_dataloader, optimizer, scheduler, device)
        accuracy, report = evaluate(model, val_dataloader, device)
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(report)

Epoch 1/20
Validation Accuracy: 0.8645
              precision    recall  f1-score   support

           0     0.8832    0.8318    0.8567      1118
           1     0.8488    0.8956    0.8715      1178

    accuracy                         0.8645      2296
   macro avg     0.8660    0.8637    0.8641      2296
weighted avg     0.8655    0.8645    0.8643      2296

Epoch 2/20
Validation Accuracy: 0.8659
              precision    recall  f1-score   support

           0     0.8450    0.8873    0.8656      1118
           1     0.8877    0.8455    0.8661      1178

    accuracy                         0.8659      2296
   macro avg     0.8663    0.8664    0.8659      2296
weighted avg     0.8669    0.8659    0.8659      2296

Epoch 3/20
Validation Accuracy: 0.8576
              precision    recall  f1-score   support

           0     0.8728    0.8283    0.8499      1118
           1     0.8445    0.8854    0.8645      1178

    accuracy                         0.8576      2296
   macro av

### Guardo el Modelo

In [69]:
torch.save(model.state_dict(), "bert_classifier.pth")


### Evaluo el modelo

In [77]:
accuracy, report = evaluate(model, test_dataloader, device)


In [78]:
print(f"Test Accuracy: {accuracy:.4f}")
print(report)

Test Accuracy: 0.8550
              precision    recall  f1-score   support

           0     0.8675    0.8461    0.8567      1176
           1     0.8425    0.8643    0.8532      1120

    accuracy                         0.8550      2296
   macro avg     0.8550    0.8552    0.8549      2296
weighted avg     0.8553    0.8550    0.8550      2296



In [79]:
test_text = "El presidente es un ladron."
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print(test_text)
print(f"Predicted veracity: {sentiment}")

tensor([[ 4.6180, -5.3795]], device='cuda:0')
torch.return_types.max(
values=tensor([4.6180], device='cuda:0'),
indices=tensor([0], device='cuda:0'))
El presidente es un ladron.
Predicted veracity: Negative
